In [1]:
import os
from dotenv import load_dotenv
from autogen import UserProxyAgent, ConversableAgent, AssistantAgent, GroupChat, GroupChatManager

load_dotenv()

llm_config={"model": "gpt-4o-mini"}


In [2]:
#Product owner talk to development team and Scrum master
product_owner_message = """
Hello Development team and Scrum master here is product owner, and I have received the following message from a customer

“ I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.
First I need a login page.
Second I need profile page.
Third I need a page that can let user input their information so the app can recommend the insurance plan.”

Here is the Product Goal and Product Backlog(How many work we need to be done) for this project.
Product Goal: develop this app in two month
Product Backlog:
1. login page
2. profile page 
3. page can recommend the insurance plan

We need to working together to planning and complete the first Sprints(Duration of each Sprints is two weeks).

I will provide the total workload(Include three Product Backlog) for each task in current project, so we can have an idea about how much work we have. 
The tasks include serveral different phase, such as Requirement, Design, Programming, Testing, and Documentation Task.


Here is the 4 STAGE we need follow to complete ONE Sprints
1. Sprint Goal: Product owner and Scrum master will working together and discuss which Product Backlog we can complete in this Sprint.
                output format: Product Backlog: EX Login page 

2. Scrum Team: After set the Sprint Goal, The Development team will working on generate the estimate plan base on the Product Backlog and total workload decided in the Sprint Goal step. 
               Every day Product owner, Scrum master and Development team will have a short meeting call Daily Scrum. 
               The puspose of this meeting is to inspect progress toward the Sprint Goal and adapt the Sprint Backlog as necessary, adjusting the upcoming planned work.

3. Sprint review: After Scrum Team step, whole team (Development Team, Product owner, Scrum master) will review the estimate plan get from the Scrum Team step . 
                  The purpose of the Sprint Review is to inspect the outcome of the Sprint and determine future adaptations. 
                  The Scrum Team presents the results of their work to key stakeholders and progress toward the Product Goal is discussed.
                  output format: Sprint Review: (review content)

4. Sprint Retrospective: After Sprints Rreview step, whole team (Development Team, Product owner, Scrum master) will do the Sprint Retrospective. 
                         The purpose of the Sprint Retrospective is to plan ways to increase quality and effectiveness.
                         output format: Sprint Retrospective: (output content)


Here is the total workload for each task in corrent project
Note: We can not complete everything in one Sprints, we will need serveral Sprints to complete all workload, but here we just complete first Sprint.

Requirement task
                                  Amount of work      Productivity Rate
Write requirements                156 req             5 req/hour
Review requirement            
    Preparation for review        156 req             8 req/hour
    Review Meeting                156 req             8 req/hour
Rework                            97 defects          5 defects/hour

///////////////////////////////////////////////////////////////////////

Design task 
                                  Amount of work      Productivity Rate
Write Design Document (DD)        134 pages           5 pages/hour
Review DD            
    Preparation for DD            134 req             3 pages/hour
    Review DD                     134 req             6 pages/hour
Rework                            88 defects          5 defects/hour


///////////////////////////////////////////////////////////////////////

Programming task
                                  Amount of work      Productivity Rate
Write code                        3945 SLOC           5 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    187 test case       3 test case/hour
    Fix Found Defects             155 defects         5 defects/day
    Test Fixed Defects            155 defects         10 defects/day
Code Inspection          
    Preparation for review        3945 SLOC           125 SLOC/hour
    Review Meeting                3945 SLOC           200 SLOC/hour
Rework                            314 defects         3 defects/hour

///////////////////////////////////////////////////////////////////////

Testing task
                                  Amount of work      Productivity Rate
Write test plan (TP)              203 pages           6 pages/day
Review TP
    Preparation for TP            203 pages           2 pages/hour
    Review TP Meeting             203 pages           6 pages/hour
    Rework                        102 defects         5 defects/day
Execute TP (test cases)           310 test case       6 test case/day
Fix Found Defects                 185 defects         12 defects/day

///////////////////////////////////////////////////////////////////////

Documentation task
                                  Amount of work      Productivity Rate
User Documentation                141 pages           5 pages/hour
Review UD           
    Preparation for review        141 pages           4 pages/hour
    Review Meeting                141 pages           10 pages/hour
Rework                            203 defects         10 defects/hour

////////////////////////////////////////////////////////////////////////

Please every engineer only working on your part and give me your estimate about your part.
Follow my total workload I provide with each task and do not estimate other engineer job.
Requirement engineer working on estimate Requirement task plan.
System engineer working on estimate Design task plan.
Software engineer working on estimate Programming task plan.
Testing engineer working on estimate Testing task plan.
Documentation engineer working on estimate Documentation task plan.
"""

In [3]:
# Agent definition
product_owner = ConversableAgent(
    name="product_owner",
    llm_config=llm_config,
    system_message=(
        "You are a project manager. Your task is to receive the customer's request, and send the message to Development Team and Scrum master."
        "Working together and we need to complete this Sprints at the end of the conversation"
        "Ask each engineer report the task they complete"
        "Ask Scrum Master which Product Backlog we can complete in this Sprint"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
    )
)

scrum_master = ConversableAgent(
    name="scrum_master",
    llm_config=llm_config,
    system_message=(
        "You are a scrum master. "
        "Your task is to help Development Team and Product owner to complete this Sprint(Sprint Goal -> Scrum Team -> Sprint review -> Sprint Retrospective)"
        "Make sure everything is on the right track and everyone is reponsible for their work"
        "Such as when Development Team provide their plan from the Sprint, you can review the plan and give some advise to them"
        "we need to complete this Sprint at the end of the conversation"
        "report which Product Backlog team can complete in this Sprint"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your final Sprint report"
        "Completed Task: ex login page"
    )
)

requirement_engineer = ConversableAgent(
    name="requirement_engineer",
    llm_config=llm_config,
    system_message=(
        "You are part of the Development team"
        "You are a requirement engineer. You ONLY work on REQUIREMENTS tasks. "
        "you will join the Daily Scrum with Scrum master and product_owner to discuss your daily work"
        "Based on the insurance recommendation app project total workload, provide the requirements work you can complete and report daily until last day of this Sprint. "
        "Consider: user input forms, recommendation logic requirements, explanation features, mobile responsiveness requirements. "
        "Use realistic estimates based on project complexity."
        "if you have any problem you can find Scrum Master, he/she can help you"
        "we need to complete this Sprint at the end of the conversation"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your daily report(Total is 10 days)"
        "Report how much work you complete in this Sprint"

        "Provide your response in this EXACT format: "
        "**Requirement Engineer**: "
        "Date: ex 6/23 Monday"
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write requirements          | [your estimate]     | [your rate]         | "
        "| Review requirements         |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
    )
)

system_engineer = ConversableAgent(
    name="system_engineer",
    llm_config=llm_config,
    system_message=(
        "You are part of the Development team"
        "You are a system engineer. You ONLY work on system tasks. "
        "you will join the Daily Scrum with Scrum master and product_owner to discuss your daily work"
        "Based on the insurance recommendation app project total workload, provide the system work you can complete and report daily until last day of this Sprint. "
        "Consider: system architecture, database design, API design, recommendation algorithm design, security architecture. "
        "Use realistic estimates based on project complexity."
        "if you have any problem you can find Scrum Master, he/she can help you"
        "we need to complete this Sprint at the end of the conversation"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your daily report(Total is 10 days)"
        "Report how much work you complete in this Sprint"

        "Provide your response in this EXACT format: "
        "**System Engineer**: "
        "Date: ex 6/23 Monday"
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write Design Document (DD)  | [your estimate]     | [your rate]         | "
        "| Review DD                   |                     |                     | "
        "|   Preparation for DD        | [your estimate]     | [your rate]         | "
        "|   Review DD                 | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
    )
)

software_engineer = ConversableAgent(
    name="software_engineer",
    llm_config=llm_config,
    system_message=(
        "You are part of the Development team"
        "You are a software engineer. You ONLY work on programming tasks. "
        "you will join the Daily Scrum with Scrum master and product_owner to discuss your daily work"
        "Based on the insurance recommendation app project total workload, provide the programming work you can complete and report daily until last day of this Sprint. "
        "Consider: frontend (forms, UI), backend (APIs, business logic), database integration, recommendation engine, mobile optimization. "
        "Use realistic estimates based on project complexity."
        "if you have any problem you can find Scrum Master, he/she can help you"
        "we need to complete this Sprint at the end of the conversation"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your daily report(Total is 10 days)"
        "Report how much work you complete in this Sprint"

        "Provide your response in this EXACT format: "
        "**Software Engineer**: "
        "Date: ex 6/23 Monday"
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write code                  | [your estimate]     | [your rate]         | "
        "| Unit Testing                |                     |                     | "
        "|   Prepare/Execute Test Cases| [your estimate]     | [your rate]         | "
        "|   Fix Found Defects         | [your estimate]     | [your rate]         | "
        "|   Test Fixed Defects        | [your estimate]     | [your rate]         | "
        "| Code Inspection             |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
    )
)

testing_engineer = ConversableAgent(
    name="testing_engineer",
    llm_config=llm_config,
    system_message=(
        "You are part of the Development team"
        "You are a testing engineer. You ONLY work on testing tasks. "
        "you will join the Daily Scrum with Scrum master and product_owner to discuss your daily work"
        "Based on the insurance recommendation app project total workload, provide the testing work you can complete and report daily until last day of this Sprint. "
        "Consider: functional testing, UI testing, mobile testing, recommendation accuracy testing, security testing, performance testing. "
        "Use realistic estimates based on project complexity."
        "if you have any problem you can find Scrum Master, he/she can help you"
        "we need to complete this Sprint at the end of the conversation"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your daily report(Total is 10 days)"
        "Report how much work you complete in this Sprint"

        "Provide your response in this EXACT format: "
        "**Testing Engineer**: "
        "Date: ex 6/23 Monday"
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write test plan (TP)        | [your estimate]     | [your rate]         | "
        "| Review TP                   |                     |                     | "
        "|   Preparation for TP        | [your estimate]     | [your rate]         | "
        "|   Review TP Meeting         | [your estimate]     | [your rate]         | "
        "|   Rework                    | [your estimate]     | [your rate]         | "
        "| Execute TP (test cases)     | [your estimate]     | [your rate]         | "
        "| Fix Found Defects           | [your estimate]     | [your rate]         | "
    )
)

documentation_engineer = ConversableAgent(
    name="documentation_engineer",
    llm_config=llm_config,
    system_message=(
        "You are part of the Development team"
        "You are a documentation engineer. You ONLY work on documentation tasks. "
        "you will join the Daily Scrum with Scrum master and product_owner to discuss your daily work"
        "Based on the insurance recommendation app project total workload, provide the documentation work you can complete and report daily until last day of this Sprint. "
        "Consider: user manual, help documentation, FAQ, admin documentation, API documentation. "
        "Use realistic estimates based on project complexity."
        "if you have any problem you can find Scrum Master, he/she can help you"
        "we need to complete this Sprint at the end of the conversation"
        "Today is 6/23 Monday"
        "This Sprint will end at 7/4 Friday"
        "NO ONE WORKING DURING WEEKEND"
        "use the format I provide here for your daily report(Total is 10 days)"
        "Report how much work you complete in this Sprint"

        "Provide your response in this EXACT format: "
        "**Documentation Engineer**: "
        "Date: ex 6/23 Monday"
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| User Documentation          | [your estimate]     | [your rate]         | "
        "| Review UD                   |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
    )
)

In [4]:
groupchat = GroupChat(
    agents=[
        product_owner,
        scrum_master,
        requirement_engineer,
        system_engineer,
        software_engineer,
        testing_engineer,
        documentation_engineer
    ],
    messages=[],
    max_round=50,
    speaker_selection_method="round_robin"
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

product_owner.initiate_chat(
    manager,
    message=product_owner_message 
)
final_outputs = manager.run()
print(final_outputs)

product_owner (to chat_manager):


Hello Development team and Scrum master here is product owner, and I have received the following message from a customer

“ I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.
First I need a login page.
Second I need profile page.
Third I need a page that can let user input their information so the app can recommend the insurance plan.”

Here is the Product Goal and Product Backlog(How many work we need to be done) for this project.
Product Goal: develop this app in two month
Product Backlog:
1. login page
2. profile page 
3. page can recommend the insurance plan

We need to working together to plann